<a href="https://colab.research.google.com/github/Tavo826/DataScience/blob/main/HandwrittenDigitRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se implementará un reconocedor usando el conjunto de datos de MNIST, usando  una rede neuronal convolucional

El MNIST contiene 60000 imágenes de entrenamiento de números desde el 0 al 9 y 10000 imágenes de prueba, representadas como una matriz 28x28 en la que cada celda contiene un valir de pixel en escala de grises

In [ ]:
import os
import numpy as  np
import matplotlib.pyplot as plt

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K

(x_train, y_train), (x_test, y_test) = mnist.load_data()

os.chdir('/content/drive/MyDrive/Colab Notebooks/Data Science/Reconocimiento de dígitos escritos a mano')

print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


### Preprocesando los datos

La dimensión del conjunto de datos es (60000,28,28) y la CNN requiere una dimensión más de modo que quede (60000,28,28,1)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

#Convirtiendo los vectores de las clases en matrices de clases binarias
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#Normalizando
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print(x_train.shape, y_train.shape)

(60000, 28, 28, 1) (60000, 10)


### Creando el modelo

Se utiliza el optimizador Adadelta

In [ ]:
batch_size = 128
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

### Entrenando el modelo

Al finalizar se guarda el modelo

In [ ]:
hist = model.fit(x_train, y_train,
                 batch_size=batch_size, epochs=epochs, verbose=10, 
                 validation_data=(x_test, y_test))

model.save('mnist.h5')

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


### Evaluando el modelo



In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes, normalize=False,
                          title=None, cmap=plt.cm.Blues):
  
  if not title:
    if normalize:
      title = 'Normalized confusion matrix'
    else:
      title = 'Confusion matrix, without normalization'
  
  #Calulando la matriz de confusión
  cm = confusion_matrix(y_true, y_pred)
  print(cm)
  #Solo se usan las etiquetas que aparecen en los datos
  #classes = classes[unique_labels(y_true, y_pred)]
  if normalize: cm = 100 * cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

  fig, ax = plt.subplots()
  im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
  ax.figure.colorbar(im, ax=ax)

  #mostrando las etiquetas
  ax.set(xticks = np.arange(cm.shape[1]),
         yticks = np.arange(cm.shape[0]),
         xticklabels = classes, yticklabels=classes, 
         title = title,
         ylabel = 'True label',
         xlabel = 'Predicted label')
  
  #rotando los ejes
  plt.setp(ax.get_xticklabels(), rotation=45, ha='right',
           rotation_mode='anchor')
  
  #Se recorren las dimensiones de los datos y se crean las anotaciones
  fmt = '.1f' if normalize else 'd'
  thresh = cm.max() / 2
  for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
      s = format(cm[i, j],'.1f') + "$\pm$" + format(np.zeros(cm.shape)[i, j],'.1f')
      ax.text(j, i, s,ha="center", va="center",
              color="white" if cm[i, j] > thresh else "black",fontsize=12)
  fig.tight_layout()
  return ax

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Test loss:  0.8953418135643005
Test accuracy:  0.8228999972343445


### Creando una GUI para predecir (no funciona en colab)

In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
from PIL import ImageGrab, Image
import numpy as np

model = load_model('mnist.h5')

def predict_digit(img):
  #modificando el tamaño de la imagen
  img = img.resize((28,28))
  #Convirtiendo a escala de grises
  img = img.convert('L')
  img = np.array(img)
  #Escalando
  img = img.reshape(1,28,28,1)
  img = img/255.0
  #Prediciendo las clases
  res = model.predict([img])[0]
  return np.argmax(res), max(res)

class App(tk.Tk):
  def __init__(self):
    tk.Tk.__init__(self)

    self.x = self.y = 0

    #Creando elementos
    self.canvas = tk.Canvas(self, width=300, height=300, bg='white', cursor='cross')
    elf.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
    self.classify_btn = tk.Button(self, text='Recognise', command=self.classify_handwriting)
    self.button_clear = tk.Button(self, text='Clear', command=self.clear_all)

    #Estructura de la grilla
    self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
    self..grid(row=0, column=1, pady=2, padx=2)
    self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
    self.button_clear.grid(row=1, column=0, pady=2)

    #self.canvas.bind("<Motion>", self.start_pos)
    self.canvas.bind("<B1-Motion>", self.draw_lines)

  def clear_all(self):
    self.canvas.delete('all')

  def classify_handwriting(self):
    HWND = self.canvas.winfo_id()
    #rect = win32gui.GetWindowRect(HWND)  -> Coordenadas del canvas
    im = ImageGrab.grab(rect)

    digit, acc = predict_digit(im)
    self.label.configure(text= str(digit) + ', ' + str(int(acc*100)) + '%')

  def draw_lines(self, event):
    self.x = event.x
    self.y = event.y
    r = 8
    self.canvas.create_oval(self.x-r, self.y-r, self.x+r, self.y+r, fill='black')

app = App()
mainloop()